In [2]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
import random
from collections import Counter

from generator import generateCliffordCircuit

In [3]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [4]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['Z', 'Z', 'I', 'H', 'X'], ['CNOT_T', 'I', 'I', 'CNOT_C', 'I'], ['X', 'I', 'H', 'I', 'Z'], ['I', 'I', 'I', 'CNOT_T', 'CNOT_C'], ['X', 'S', 'H', 'H', 'H']]
     ┌───┐ ░ ┌───┐ ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ Z ├─░─┤ X ├─░─┤ X ├─░───────░─┤ X ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤
q_1: ┤ Z ├─░───┼───░─┤ I ├─░───────░─┤ S ├
     ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤
q_2: ┤ I ├─░───┼───░─┤ H ├─░───────░─┤ H ├
     ├───┤ ░   │   ░ ├───┤ ░ ┌───┐ ░ ├───┤
q_3: ┤ H ├─░───■───░─┤ I ├─░─┤ X ├─░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_4: ┤ X ├─░───────░─┤ Z ├─░───■───░─┤ H ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [5]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")

In [6]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('YYXYX')

In [28]:
def params_list(operation_set, pauli_set=["X","Y","Z","I"]):
    '''This function generates a dictionary of parameters for the given operation set.
    Inputs: operation_set - A list of operations.
            pauli_set - A list of pauli operators.
    Outputs: params - A dictionary of parameters, with each set to 0.'''

    params = {}
    for operation in operation_set:
        if operation == "CNOT_C" or operation == "CNOT_T":
            print("CNOT_C or CNOT_T")
            for pauli1 in pauli_set:
                for pauli2 in pauli_set:
                    pauli = pauli1+pauli2
                    params[("CNOT", pauli)] = 0
        for pauli in pauli_set:
            params[(operation, pauli)] = 0
    return params

print(params_list(doubleGateSet))

CNOT_C or CNOT_T
CNOT_C or CNOT_T
{('CNOT', 'XX'): 0, ('CNOT', 'XY'): 0, ('CNOT', 'XZ'): 0, ('CNOT', 'XI'): 0, ('CNOT', 'YX'): 0, ('CNOT', 'YY'): 0, ('CNOT', 'YZ'): 0, ('CNOT', 'YI'): 0, ('CNOT', 'ZX'): 0, ('CNOT', 'ZY'): 0, ('CNOT', 'ZZ'): 0, ('CNOT', 'ZI'): 0, ('CNOT', 'IX'): 0, ('CNOT', 'IY'): 0, ('CNOT', 'IZ'): 0, ('CNOT', 'II'): 0, ('CNOT_C', 'X'): 0, ('CNOT_C', 'Y'): 0, ('CNOT_C', 'Z'): 0, ('CNOT_C', 'I'): 0, ('CNOT_T', 'X'): 0, ('CNOT_T', 'Y'): 0, ('CNOT_T', 'Z'): 0, ('CNOT_T', 'I'): 0}


In [8]:
def remove_negatives(pauli):
    '''This function takes a pauli operator and removes the negative sign from it.'''

    if pauli.to_label()[0] == "-":
        return qk.quantum_info.Pauli(pauli.to_label()[1:])
    else:
        return pauli

In [9]:
def pauli_hist(circuit, pauli_initial, params_list=params_list(singleGateSet)):
    '''This function computes the histogram of pauli operators after the circuit has been applied.
    Inputs: circuit - A Qiskit Circuit, Clifford, or Gate object.
            pauli_initial - A Qiskit Pauli object.
            params_list - A dictionary of parameters for the circuit.
    Outputs: pauli_counts - A dictionary of pauli operators and their counts.'''

    pauli = pauli_initial

    for op_instructions in circuit.data:
        if op_instructions.operation.name.upper() == "ID" or op_instructions.operation.name.upper() == "I" or op_instructions.operation.name.upper() == "BARRIER":
            print("skipped ", op_instructions.operation.name.upper(), " operation")
            break
        elif op_instructions.operation.name.upper() == "CX" or op_instructions.operation.name.upper() == "CNOT":
            params_list[("CNOT_C", pauli.to_label()[0])] += 1
            params_list[("CNOT_T", pauli.to_label()[1])] += 1
            pauli = qk.quantum_info.Pauli(pauli.to_label()[1] + pauli.to_label()[0] + pauli.to_label()[2:])
        else:
            pauli = remove_negatives(pauli)
            params_list[(op_instructions.operation.name.upper(), pauli.to_label())] += 1
            pauli = Clifford_Permute(op_instructions.operation, pauli)

    return params_list

print(pauli_hist(qiskitCircuit, qk.quantum_info.Pauli('X')))

skipped  ID  operation
{('X', 'X'): 0, ('X', 'Y'): 0, ('X', 'Z'): 0, ('X', 'I'): 0, ('H', 'X'): 0, ('H', 'Y'): 0, ('H', 'Z'): 0, ('H', 'I'): 0, ('Z', 'X'): 2, ('Z', 'Y'): 0, ('Z', 'Z'): 0, ('Z', 'I'): 0, ('I', 'X'): 0, ('I', 'Y'): 0, ('I', 'Z'): 0, ('I', 'I'): 0, ('S', 'X'): 0, ('S', 'Y'): 0, ('S', 'Z'): 0, ('S', 'I'): 0}


In [18]:
def split_circuit_by_barrier(circuit):
    qasm = circuit.qasm()
    prelude = []
    circuits = [[]]
    for line in qasm.splitlines():
        if any([line.startswith(t) for t in ['OPENQASM', 'include', 'qreg', 'creg']]):
            prelude.append(line)
        elif line.startswith('barrier'):
            circuits.append([])
        else:
            circuits[-1].append(line)
    circuits_with_prelude = [prelude+circuit for circuit in circuits]
    for circuit_with_prelude in circuits_with_prelude:
        yield qk.QuantumCircuit.from_qasm_str('\n'.join(circuit_with_prelude))

# for subcircuit in (split_circuit_by_barrier(qiskitCircuit)):
#     print(subcircuit)

In [25]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

qiskitCircuit = transpileListToQiskitCircuit(generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet))
print(qiskitCircuit)

first_layer = ""
for item in list(split_circuit_by_barrier(qiskitCircuit))[0].data:
    print(item.operation)
    first_layer += item.operation.name
    print(first_layer)

# print(pauli_hist(qiskitCircuit, qk.quantum_info.Pauli('X')))

     ┌───┐ ░ ┌───┐ ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ S ├─░─┤ X ├─░─┤ Z ├─░───────░─┤ H ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░ ┌───┐ ░ ├───┤
q_1: ┤ H ├─░───■───░─┤ Z ├─░─┤ X ├─░─┤ S ├
     ├───┤ ░       ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_2: ┤ Z ├─░───────░─┤ Z ├─░───┼───░─┤ X ├
     ├───┤ ░       ░ ├───┤ ░   │   ░ ├───┤
q_3: ┤ X ├─░───────░─┤ Z ├─░───■───░─┤ S ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_4: ┤ S ├─░───────░─┤ S ├─░───────░─┤ S ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘
Instruction(name='s', num_qubits=1, num_clbits=0, params=[])
s
Instruction(name='h', num_qubits=1, num_clbits=0, params=[])
sh
Instruction(name='z', num_qubits=1, num_clbits=0, params=[])
shz
Instruction(name='x', num_qubits=1, num_clbits=0, params=[])
shzx
Instruction(name='s', num_qubits=1, num_clbits=0, params=[])
shzxs


In [10]:
def generate_A(numrow, numcol):
    '''This function generates the A matrix for the linear system Ax=b, where x is the vector of 
    parameters for the circuit, and b is the vector of pauli counts.
    Inputs: numrow - The number of rows in the A matrix.
            numcol - The number of columns in the A matrix.
    Outputs: A - The A matrix.'''

    A = np.zeros((numrow,numcol))
    for i in range(numrow):
        for j in range(numcol):
            if i == j:
                A[i,j] = 1
            elif i < j:
                A[i,j] = 1
    return A

print(generate_A(5, 5))


[[1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]]
